In [ ]:
import time
import pandas as pd
import pandapower as pp
import numpy as np
import matplotlib.pyplot as plt
import os
from pandapower.plotting.plotly import pf_res_plotly
from pandapower.plotting.plotly import simple_plotly

import Utils.Network as Network
import Utils.Limits as Limits
import Utils.Market as Market
import Utils.utils as utils

import time

# Import Network

In [ ]:
input_path = '../Data/Input'
output_path = '../Data/Output'

network_obj = Network.Network(input_path)

feeder_colors = ['#ff7f0e', '#9467bd']
feeder_colors_after = [utils.scale_lightness(c, 0.6) for c in feeder_colors]

meaningful_days = [15, 83, 162, 241, 324]
timesteps = network_obj.get_meaningful_days_timesteps(meaningful_days)

limits_obj = Limits.Limits(network_obj)
limits_obj.load_limits("solution_limits.npy")
market_obj = Market.Market(network_obj, limits_obj)

In [ ]:
ga_obj = limits_obj.calculate_limits()

In [ ]:
network_obj.plot_P(4, limits=limits_obj.limits)

In [ ]:
0.8**5

In [ ]:
ti = time.time()
print(f"Expected execution time: {(len(network_obj.timesteps) * 4.7):.1f} seconds; minutes: {(len(network_obj.timesteps) * 4.7 / 60):.1f}")
for t in network_obj.timesteps[:100]:
    network_obj.load_time_series_at_timestep(network_obj.net, t)
    market_obj.submit_orders()
    market_obj.market_clearing()
tf = time.time()
print(f"Time taken: {(tf - ti):.1f} seconds. Average: {((tf - ti) / len(network_obj.timesteps[:100])):.3f} seconds")

In [ ]:
limits_obj.plot_limits_usage(network_obj.net, limits_obj.initial_limits)
limits_obj.plot_limits_usage(network_obj.net, limits_obj.limits)

In [ ]:
market_obj.traded_limits

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_traded_limits(market_obj):
    """
    Plot traded limits across timesteps with enhanced visualization
    
    Parameters:
        market_obj: Market object containing traded_limits list
                   Each element is a list of traded amounts for a timestep
    """
    plt.figure(figsize=(12, 6))
    
    # Convert to numpy array for vectorized operations
    timesteps = np.arange(len(market_obj.traded_limits))
    traded_limits = [np.sum(np.abs(t)) if t else 0 for t in market_obj.traded_limits]
    
    # Main plot with custom styling
    bars = plt.bar(timesteps, traded_limits, 
                   color='skyblue', 
                   edgecolor='navy',
                   linewidth=1.2,
                   alpha=0.8,
                   label='Total Traded Volume')
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        # if height > 0:
        #     plt.text(bar.get_x() + bar.get_width()/2., height,
        #             f'{height:.1f}',
        #             ha='center', va='bottom',
        #             fontsize=7)
    
    # Customize plot appearance
    plt.title('Market Clearing Volume by Timestep', pad=20, fontsize=14)
    plt.xlabel('Timestep', labelpad=10)
    plt.ylabel('Traded Volume (kW)', labelpad=10)
    # plt.xticks(timesteps)
    
    # Add grid and style
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.gca().set_axisbelow(True)
    
    plt.legend()
    plt.tight_layout()
    plt.show()

# Usage
plot_traded_limits(market_obj)